In [2]:
import matplotlib.pyplot as plt #Plotting
import numpy as np #Number manipulation
import re #String manipulation
import pandas as pd #Dataframe manipulation
import math #Math
import matplotlib #Plotting
#from __future__ import division #Math
import seaborn as sns #Better plotting
import scipy #Statistical tests
from keras.models import Sequential, Input, Model
from keras.layers import LSTM, Dense, Dropout, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
%matplotlib inline 
from ipykernel import kernelapp as app


Using TensorFlow backend.


In [ ]:
# Last 100 data of Image Stage + first 100 data of Decision Stage

In [3]:
subjects = ["subject12study.csv", "subject13study.csv", "subject14study.csv", "subject15study.csv"]
infos = ["Subject 12.csv", "Subject 13.csv", "Subject 14.csv", "Subject 15.csv"]

data_main = []
data_side = []

for i in range(len(subjects)):
    df = pd.read_csv(subjects[i], delimiter = "\t")
    info = pd.read_csv(infos[i])

    # Cleaning the data
    df = df[df["avgx"] !=0] #eyex and eyey cannot be equal to 0
    df = df[df["avgy"] !=0]
    df = df[df["avgx"] > 0] #eyex should be bigger than 0 and smaller than 1920, regarding the screen size during the study
    df = df[df["avgx"] <1920]
    df = df[df["avgy"] > 0]  
    df = df[df["avgy"] < 1080] #eyey should be bigger than 0 and smaller than 1080, according to the screen size during the study
    df = df[df["psize"] < 25] #pupil size can be max 25 and min 13
    df = df[df["psize"] >13]

    data_main.append(df)
    data_side.append(info)


/home/magus/anaconda3/envs/gym/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
datax = []

for q in range(len(data_main)):
    
    realtime = [] #realtime
    for a in data_main[q]["timestamp"]:
        try:
            d , f = a.split()
            (h, m, s) = f.split(':')
            result = float(h) * 3600 + float(m) * 60 + float(s)
            realtime.append(result)
        except:
            realtime.append(float(0))

    len(realtime)
    data_main[q]["realtime"] = realtime
    dfx = data_main[q][["realtime", "avgx", "avgy", "psize"]] #creating a new dataframe for only requiered columns
    
    datax.append(dfx)
    
    imgstart = []
    imgend =[]
    decstart =[]
    decenda = []
    decendb =[]


    for i in data_side[q]["ImgStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resulti = float(h)*3600 + float(m)*60 + float(s)
            imgstart.append(resulti)

    for i in data_side[q]["ImgEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resultie = float(h)*3600 + float(m)*60 + float(s)
            imgend.append(resultie)

    for e in data_side[q]["EmptyStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultes = float(h)*3600 + float(m)*60 + float(s)
            decstart.append(resultes)

    for e in data_side[q]["EmptyEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultee = float(h)*3600 + float(m)*60 + float(s)
            decenda.append(resultee)

    cro2beg = []
    contbeg = []
    cro1beg = []

    for c in data_side[q]["ComplexStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = c.split(":")
            result = float(h)*3600 + float(m)*60 + float(s)
            decendb.append(result)
            cro2beg.append(result-9)
            contbeg.append(result-13)
            cro1beg.append(result-14)

    #print (imgstart[1], imgend[1], decstart[1], decenda[1], decendb[1] )

    data_side[q]["Cro1Beg"] = cro1beg  #cross1(fixation)
    data_side[q]["ContBeg"] = contbeg  #context
    data_side[q]["Cro2Beg"] = cro2beg  #cross2(fixation)
    data_side[q]["ImagesBeg"] = imgstart    #images stage started
    data_side[q]["ImagesEnd"] = imgend
    data_side[q]["DecisionBeg"] = decstart   #decision stage started  (green frame around the pictures)
    data_side[q]["DecisionEnd"] = decenda


    data_side[q].sort_values("DecisionEnd", inplace=True, ascending=True) #reorganizing dataframe

    data_side[q] = data_side[q].reset_index(drop=True) #saving df

In [5]:
"""
datac[0][4][79]
for i in range(80):
    print(len(datac[0][4][i]))
"""

'\ndatac[0][4][79]\nfor i in range(80):\n    print(len(datac[0][4][i]))\n'

In [6]:
#len(datax), datax[0].shape, len(data_side), data_side[0].shape
#len(datac), len(datac[0]), len(datac[0][0])
#len(datac), len(datac[0]), len(datac[0][0]), len(datac[0][0][1])

In [7]:

datac = []

for q in range(len(datax)):
    
    cross1 = []
    context = []
    cross2 = []
    images = []
    decision=[]
    sub = []
    
    for i in range(80):
        cross1.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro1Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ContBeg"][i]))])
        context.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ContBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["Cro2Beg"][i]))])
        cross2.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro2Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesBeg"][i]))])
        images.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ImagesBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesEnd"][i]))])
        decision.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["DecisionBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["DecisionEnd"][i]))])
    
    sub.append(cross1)
    sub.append(context)
    sub.append(cross2)
    sub.append(images)
    sub.append(decision)
    
    datac.append(sub)

In [8]:
Cx = [] #choice data as left and right
for i in range(4):
    c = []
    c = data_side[i]['Choice'].tolist()
    Cx.append(c)

In [15]:
# Get the last 100 data points of the Image Stage

len(datac[0][4]), len(Cx[0])

img_data = []
dec_data = []
cdata = []


for i in range(4):
    for q in range(80):
        img_data.append(datac[i][3][q]) #Image stage data
        dec_data.append(datac[i][4][q]) #Decision stage data
        cdata.append(Cx[i][q])

nnx = []
nnc = []

for i in range(len(img_data)):
    #print(len(xdata[i]), cdata[i])
    
    if len(img_data[i]) >= 100 & len(dec_data[i]) >= 100:
        a = img_data[i][-100:] # Gets last 100 data points, use [:100] to get the first 100
        b = dec_data[i][:100] # Gets the first 100 data points
        j = a.append(b)
        #print(j.shape)
        z = j.values[:,1:]
        
        nnx.append(z)
        nnc.append(cdata[i])
    else:
        pass

In [16]:
for i in nnx:
    i[:,0] = i[:,0]/1920
    i[:,1] = i[:,1]/1080
    i[:,2] = (i[:,2]-13)/12

In [24]:
valx = nnx[:5]
valy = nnc[:5]

In [25]:
nn_x = np.stack(nnx[5:])
nn_c = np.stack(nnc[5:])
val_x = np.stack(valx)
val_y = np.stack(valy)
print(nn_x.shape, nn_c.shape, val_x.shape, val_y.shape)


(76, 200, 3) (76,) (5, 200, 3) (5,)


In [26]:
d= []
for i in nn_c:
    if i == 1:
        k = [1,0]
        d.append(k)
    elif i == 0:
        k = [0,1]
        d.append(k)

e = np.stack(d).reshape(76,2,1)
f = nn_x.reshape(76,600)
g = val_x.reshape(5,600)
f.shape

(76, 600)

In [27]:
inputs = Input(shape=(600,))

x = Dense(32, activation='relu')(inputs)
x = Dense(16, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model = Model(inputs=inputs, outputs=prediction)
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])
history = model.fit(f, nn_c, epochs=300, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 600)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                19232     
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 19,777
Trainable params: 19,777
Non-trainable params: 0
_________________________________________________________________
Train on 60 samples, validate on 16 samples
Epoch 1/300
 - 1s - loss: 0.7357 - binary_accuracy: 0.5000 - val_loss: 0.6617 - val_binary_accuracy: 0.4375
Epoch 2/300
 - 0s - loss: 0.6938 - binary_accuracy: 0.5167 - val_loss: 0.6820 - val_binary_accuracy: 0.5625
Epoch 3/30

Epoch 68/300
 - 0s - loss: 0.2633 - binary_accuracy: 0.9333 - val_loss: 0.1950 - val_binary_accuracy: 0.9375
Epoch 69/300
 - 0s - loss: 0.2590 - binary_accuracy: 0.9333 - val_loss: 0.1305 - val_binary_accuracy: 1.0000
Epoch 70/300
 - 0s - loss: 0.2567 - binary_accuracy: 0.9333 - val_loss: 0.1907 - val_binary_accuracy: 0.9375
Epoch 71/300
 - 0s - loss: 0.2532 - binary_accuracy: 0.9333 - val_loss: 0.1289 - val_binary_accuracy: 1.0000
Epoch 72/300
 - 0s - loss: 0.2521 - binary_accuracy: 0.9333 - val_loss: 0.1839 - val_binary_accuracy: 0.9375
Epoch 73/300
 - 0s - loss: 0.2468 - binary_accuracy: 0.9333 - val_loss: 0.1262 - val_binary_accuracy: 1.0000
Epoch 74/300
 - 0s - loss: 0.2459 - binary_accuracy: 0.9333 - val_loss: 0.1871 - val_binary_accuracy: 0.9375
Epoch 75/300
 - 0s - loss: 0.2455 - binary_accuracy: 0.9333 - val_loss: 0.1226 - val_binary_accuracy: 1.0000
Epoch 76/300
 - 0s - loss: 0.2459 - binary_accuracy: 0.9333 - val_loss: 0.1882 - val_binary_accuracy: 0.9375
Epoch 77/300
 - 0s 

Epoch 143/300
 - 0s - loss: 0.1476 - binary_accuracy: 0.9500 - val_loss: 0.0671 - val_binary_accuracy: 1.0000
Epoch 144/300
 - 0s - loss: 0.1515 - binary_accuracy: 0.9667 - val_loss: 0.1374 - val_binary_accuracy: 0.9375
Epoch 145/300
 - 0s - loss: 0.1473 - binary_accuracy: 0.9500 - val_loss: 0.0661 - val_binary_accuracy: 1.0000
Epoch 146/300
 - 0s - loss: 0.1558 - binary_accuracy: 0.9500 - val_loss: 0.1402 - val_binary_accuracy: 0.9375
Epoch 147/300
 - 0s - loss: 0.1479 - binary_accuracy: 0.9500 - val_loss: 0.0643 - val_binary_accuracy: 1.0000
Epoch 148/300
 - 0s - loss: 0.1479 - binary_accuracy: 0.9667 - val_loss: 0.1267 - val_binary_accuracy: 0.9375
Epoch 149/300
 - 0s - loss: 0.1395 - binary_accuracy: 0.9667 - val_loss: 0.0664 - val_binary_accuracy: 1.0000
Epoch 150/300
 - 0s - loss: 0.1404 - binary_accuracy: 0.9667 - val_loss: 0.1209 - val_binary_accuracy: 0.9375
Epoch 151/300
 - 0s - loss: 0.1343 - binary_accuracy: 0.9667 - val_loss: 0.0649 - val_binary_accuracy: 1.0000
Epoch 152/

Epoch 218/300
 - 0s - loss: 0.0695 - binary_accuracy: 0.9833 - val_loss: 0.0583 - val_binary_accuracy: 1.0000
Epoch 219/300
 - 0s - loss: 0.0687 - binary_accuracy: 0.9833 - val_loss: 0.0515 - val_binary_accuracy: 1.0000
Epoch 220/300
 - 0s - loss: 0.0679 - binary_accuracy: 0.9833 - val_loss: 0.0624 - val_binary_accuracy: 0.9375
Epoch 221/300
 - 0s - loss: 0.0675 - binary_accuracy: 0.9833 - val_loss: 0.0422 - val_binary_accuracy: 1.0000
Epoch 222/300
 - 0s - loss: 0.0692 - binary_accuracy: 0.9833 - val_loss: 0.0963 - val_binary_accuracy: 0.9375
Epoch 223/300
 - 0s - loss: 0.0792 - binary_accuracy: 0.9833 - val_loss: 0.0579 - val_binary_accuracy: 1.0000
Epoch 224/300
 - 0s - loss: 0.1467 - binary_accuracy: 0.9500 - val_loss: 0.2176 - val_binary_accuracy: 0.9375
Epoch 225/300
 - 0s - loss: 0.1723 - binary_accuracy: 0.9167 - val_loss: 0.0609 - val_binary_accuracy: 1.0000
Epoch 226/300
 - 0s - loss: 0.1476 - binary_accuracy: 0.9500 - val_loss: 0.0651 - val_binary_accuracy: 0.9375
Epoch 227/

Epoch 293/300
 - 0s - loss: 0.0355 - binary_accuracy: 1.0000 - val_loss: 0.0384 - val_binary_accuracy: 1.0000
Epoch 294/300
 - 0s - loss: 0.0352 - binary_accuracy: 1.0000 - val_loss: 0.0381 - val_binary_accuracy: 1.0000
Epoch 295/300
 - 0s - loss: 0.0348 - binary_accuracy: 1.0000 - val_loss: 0.0389 - val_binary_accuracy: 1.0000
Epoch 296/300
 - 0s - loss: 0.0346 - binary_accuracy: 1.0000 - val_loss: 0.0368 - val_binary_accuracy: 1.0000
Epoch 297/300
 - 0s - loss: 0.0341 - binary_accuracy: 1.0000 - val_loss: 0.0386 - val_binary_accuracy: 1.0000
Epoch 298/300
 - 0s - loss: 0.0337 - binary_accuracy: 1.0000 - val_loss: 0.0351 - val_binary_accuracy: 1.0000
Epoch 299/300
 - 0s - loss: 0.0333 - binary_accuracy: 1.0000 - val_loss: 0.0395 - val_binary_accuracy: 1.0000
Epoch 300/300
 - 0s - loss: 0.0329 - binary_accuracy: 1.0000 - val_loss: 0.0337 - val_binary_accuracy: 1.0000


In [29]:
inputs = Input(shape=(600,))

x = Dense(8, activation='relu')(inputs)
x = Dense(4, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model2 = Model(inputs=inputs, outputs=prediction)
model2.summary()
model2.compile(optimizer='adam', loss='mse', metrics=['binary_accuracy'])
history = model2.fit(f, nn_c, epochs=300, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 600)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 4808      
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 5         
Total params: 4,849
Trainable params: 4,849
Non-trainable params: 0
_________________________________________________________________
Train on 60 samples, validate on 16 samples
Epoch 1/300
 - 0s - loss: 0.2433 - binary_accuracy: 0.5167 - val_loss: 0.2357 - val_binary_accuracy: 0.4375
Epoch 2/300
 - 0s - loss: 0.2354 - binary_accuracy: 0.5333 - val_loss: 0.2203 - val_binary_accuracy: 0.5625
Epoch 3/300


Epoch 68/300
 - 0s - loss: 0.0700 - binary_accuracy: 0.9333 - val_loss: 0.0398 - val_binary_accuracy: 0.9375
Epoch 69/300
 - 0s - loss: 0.0693 - binary_accuracy: 0.9333 - val_loss: 0.0393 - val_binary_accuracy: 0.9375
Epoch 70/300
 - 0s - loss: 0.0686 - binary_accuracy: 0.9333 - val_loss: 0.0389 - val_binary_accuracy: 0.9375
Epoch 71/300
 - 0s - loss: 0.0679 - binary_accuracy: 0.9333 - val_loss: 0.0384 - val_binary_accuracy: 0.9375
Epoch 72/300
 - 0s - loss: 0.0673 - binary_accuracy: 0.9333 - val_loss: 0.0379 - val_binary_accuracy: 0.9375
Epoch 73/300
 - 0s - loss: 0.0666 - binary_accuracy: 0.9333 - val_loss: 0.0374 - val_binary_accuracy: 0.9375
Epoch 74/300
 - 0s - loss: 0.0660 - binary_accuracy: 0.9333 - val_loss: 0.0369 - val_binary_accuracy: 0.9375
Epoch 75/300
 - 0s - loss: 0.0654 - binary_accuracy: 0.9333 - val_loss: 0.0365 - val_binary_accuracy: 0.9375
Epoch 76/300
 - 0s - loss: 0.0648 - binary_accuracy: 0.9333 - val_loss: 0.0362 - val_binary_accuracy: 0.9375
Epoch 77/300
 - 0s 

Epoch 143/300
 - 0s - loss: 0.0415 - binary_accuracy: 0.9667 - val_loss: 0.0230 - val_binary_accuracy: 0.9375
Epoch 144/300
 - 0s - loss: 0.0414 - binary_accuracy: 0.9667 - val_loss: 0.0229 - val_binary_accuracy: 0.9375
Epoch 145/300
 - 0s - loss: 0.0412 - binary_accuracy: 0.9667 - val_loss: 0.0228 - val_binary_accuracy: 0.9375
Epoch 146/300
 - 0s - loss: 0.0411 - binary_accuracy: 0.9667 - val_loss: 0.0228 - val_binary_accuracy: 0.9375
Epoch 147/300
 - 0s - loss: 0.0409 - binary_accuracy: 0.9667 - val_loss: 0.0228 - val_binary_accuracy: 0.9375
Epoch 148/300
 - 0s - loss: 0.0408 - binary_accuracy: 0.9667 - val_loss: 0.0228 - val_binary_accuracy: 0.9375
Epoch 149/300
 - 0s - loss: 0.0406 - binary_accuracy: 0.9667 - val_loss: 0.0227 - val_binary_accuracy: 0.9375
Epoch 150/300
 - 0s - loss: 0.0405 - binary_accuracy: 0.9667 - val_loss: 0.0226 - val_binary_accuracy: 0.9375
Epoch 151/300
 - 0s - loss: 0.0404 - binary_accuracy: 0.9667 - val_loss: 0.0225 - val_binary_accuracy: 0.9375
Epoch 152/

Epoch 218/300
 - 0s - loss: 0.0358 - binary_accuracy: 0.9667 - val_loss: 0.0204 - val_binary_accuracy: 0.9375
Epoch 219/300
 - 0s - loss: 0.0358 - binary_accuracy: 0.9667 - val_loss: 0.0204 - val_binary_accuracy: 0.9375
Epoch 220/300
 - 0s - loss: 0.0357 - binary_accuracy: 0.9667 - val_loss: 0.0203 - val_binary_accuracy: 0.9375
Epoch 221/300
 - 0s - loss: 0.0357 - binary_accuracy: 0.9667 - val_loss: 0.0203 - val_binary_accuracy: 0.9375
Epoch 222/300
 - 0s - loss: 0.0357 - binary_accuracy: 0.9667 - val_loss: 0.0203 - val_binary_accuracy: 0.9375
Epoch 223/300
 - 0s - loss: 0.0356 - binary_accuracy: 0.9667 - val_loss: 0.0203 - val_binary_accuracy: 0.9375
Epoch 224/300
 - 0s - loss: 0.0356 - binary_accuracy: 0.9667 - val_loss: 0.0203 - val_binary_accuracy: 0.9375
Epoch 225/300
 - 0s - loss: 0.0356 - binary_accuracy: 0.9667 - val_loss: 0.0203 - val_binary_accuracy: 0.9375
Epoch 226/300
 - 0s - loss: 0.0356 - binary_accuracy: 0.9667 - val_loss: 0.0203 - val_binary_accuracy: 0.9375
Epoch 227/

Epoch 293/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0199 - val_binary_accuracy: 0.9375
Epoch 294/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0199 - val_binary_accuracy: 0.9375
Epoch 295/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0199 - val_binary_accuracy: 0.9375
Epoch 296/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0199 - val_binary_accuracy: 0.9375
Epoch 297/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0199 - val_binary_accuracy: 0.9375
Epoch 298/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0200 - val_binary_accuracy: 0.9375
Epoch 299/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0200 - val_binary_accuracy: 0.9375
Epoch 300/300
 - 0s - loss: 0.0344 - binary_accuracy: 0.9667 - val_loss: 0.0199 - val_binary_accuracy: 0.9375


In [30]:
s = model.predict(g)

In [31]:
model.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 1s 534us/step


[0.03078208491206169, 1.0]

In [32]:
model2.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 0s 438us/step


[0.000895645294804126, 1.0]

In [35]:
for i in range(5):
    if s[i] > 0.5:
        print(1, val_y[i])
    else:
        print(0, val_y[i])

1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
